In [1]:
import pickle
import os
import re
import pandas as pd
import numpy as np

In [2]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument

C:\Users\morad\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
#the directory of pickle files
usr_dir = r'C:\GH-Dataset\Git_dt_roles\users'

In [4]:
#the directory of pickle files
issue_dir = r'C:\GH-Dataset\Git_dt_roles\issues'

In [7]:
#Load files in pickle format. There is a seperate pickle file per each developer.
def open_file(file_path):
    with open(file_path, "rb") as picklefile:
        reader = pickle.load(picklefile)
    return(reader)

In [6]:
#Seprate lines in text with \n only (replace other newline tags with \n)
def newline(text):
    return '\n'.join([p for p in re.split(r'\\n|\\r|\\n\\n|\\r\\n|\r\n', text) if len(p) > 0])

In [7]:
#merge a list of readme files in a single string which are seperated with \n 
def merg_RM(RM):
    return '\n '.join(RM)

In [8]:
#merge to texts in one single text
def merg_text(txt1,txt2):
    return txt1 +"\n "+txt2

In [10]:
#issue_path = os.path.join(directory, "4gus71n.pickle")

In [5]:
user_path = os.path.join(usr_dir, "josephmtinangi.pickle")

In [8]:
user_reader = open_file(user_path) 

In [12]:
user_reader

{'name': 'Joseph Mtinangi',
 'u_id': 'josephmtinangi',
 'date': datetime.datetime(2015, 1, 19, 5, 31, 17),
 'activity': 'Wed, 19 May 2021 07:22:04 GMT',
 'url': 'https://github.com/josephmtinangi',
 'email': 'josephmtinangi@gmail.com',
 'location': 'Dodoma, Tanzania',
 'blog': 'https://josephmtinangi.github.io',
 'bio': 'Software developer, I like to contribute to the open source community and use modern software development approaches & tools',
 'repository': {0: {'repo_name': 'a-beautiful-htmlcss-website',
   'readme': b'# A beautiful html css website\n\nJust strengthening my css skills\n',
   'topics': 'none',
   'about': None,
   'stars': 0,
   'languages': {'CSS': 56.00311041990669, 'HTML': 43.99688958009331},
   'total_commits': 18,
   'author_commits': 18,
   'author_contribution': 100.0,
   'files': {'md': 1, 'css': 9, 'html': 13, 'jpg': 11}},
  1: {'repo_name': 'a-simple-restful-api-using-php-and-codeigniter',
   'readme': 'none',
   'topics': 'none',
   'about': None,
   'sta

In [17]:
for i in user_reader['repository'][0]['languages']:
    print(i)

CSS
HTML


In [22]:
issue_reader = open_file(issue_path) 

In [37]:
for j in issue_reader['issues'][37]:
    print(j)

0
1
2


In [35]:
if (issue_reader['issues'][7]):
    print("yes")

In [26]:
issue_reader

{'name': 'Joseph Mtinangi',
 'u_id': 'josephmtinangi',
 'issues': {0: {},
  1: {},
  2: {},
  3: {},
  4: {},
  5: {},
  6: {0: {'title': 'Basic jquery form validation',
    'body': 'Finish experimenting with the feature\n'}},
  7: {},
  8: {},
  9: {},
  10: {},
  11: {},
  12: {},
  13: {},
  14: {},
  15: {},
  16: {},
  17: {},
  18: {},
  19: {},
  20: {},
  21: {},
  22: {},
  23: {},
  24: {},
  25: {},
  26: {},
  27: {},
  28: {},
  29: {},
  30: {},
  31: {},
  32: {0: {'title': 'Create README.md file',
    'body': 'Create README.md file to help people interested in this repository understand the project.'}},
  33: {},
  34: {},
  35: {},
  36: {},
  37: {0: {'title': 'More information in README file is required',
    'body': 'Information like what is this project about, how to install, how to contribute, who are the contributors etc.'},
   1: {'title': 'PHP Notice:  Undefined index: host in project\\location\\config\\database.php',
    'body': 'Whatever artisan command I run

In [10]:
#define the final DataFrame format
Corpus = pd.DataFrame(columns=['Name','u_id','bio','Repo_Name','ReadMe','issues','textual_all'])

In [11]:
#loop over all files in a directory and collect required fields
for filename in os.listdir(usr_dir):
    usr_path = os.path.join(usr_dir, filename)
    reader = open_file(usr_path)
    try:
        issue_path = os.path.join(issue_dir, filename)
        issue_reader = open_file(issue_path)
    except:
        issue_reader=None
     
    bio   = newline(str(reader['bio']))
    Name = str(reader['name'])
    try:
        u_id= str(reader['u_id'])
    except:
        u_id ="none"
        
    RM = []
    repo_name=[]
    repo_tags=[]
    repo_topics=[]
    repo_text=[]
    issue_txt_all=[]
    
    for row in range(0,len(reader['repository'])):
        repo_name_tmp = str(reader['repository'][row]['repo_name'])
        repo_RM_tmp = newline(str(reader['repository'][row]['readme']))
        repo_tags_tmp = str(reader['repository'][row]['topics'])
        repo_topics_tmp=newline(str(reader['repository'][row]['about']))
        
                
        repo_name.append(repo_name_tmp)
        RM.append(repo_RM_tmp)
        repo_tags.append(repo_tags_tmp)
        repo_topics.append(repo_topics_tmp)   
        
        
        repo_text_tmp = merg_text(merg_text(repo_tags_tmp,repo_topics_tmp),merg_text(repo_name_tmp,merg_text(bio,repo_RM_tmp)))
                                  
        try:
            if((issue_reader is not None) and (issue_reader['issues'][row])):
                for j in issue_reader['issues'][row]:
                    issue_title= str(issue_reader['issues'][row][j]['title'])
                    issue_body= str(issue_reader['issues'][row][j]['body'])
                    issue_temp=merg_text(issue_title,issue_body)
                    repo_text_tmp= merg_text(repo_text_tmp,issue_temp)
                    issue_txt_all.append(issue_temp)
        except:
            print(u_id)
                                  
                                  
        repo_text.append(repo_text_tmp)
        
        
        
    text_all = merg_RM(repo_text) 
    info_all = merg_text(bio,text_all)
    
    
    Corpus = Corpus.append({'Name':Name,'u_id':u_id,'bio':bio,'Repo_Name':repo_name,'ReadMe':RM, 'issues':issue_txt_all,
                            'textual_all':info_all},ignore_index=True)
    

AaronAstle
AaronAstle
AaronAstle
akarsh
albertmolodec
AlekseyPudlo
atb00ker
beingadityak
beingadityak
beingadityak
beingadityak
beingadityak
Better-Boy
BharathKumarS
bksinn
bksinn
charlesfranciscodev
charlesfranciscodev
danielcardeenas
danielcardeenas
danielcardeenas
davidcetinkaya
DeepakJhaaa
DeepakJhaaa
DeepakJhaaa
DeepakJhaaa
DeepakJhaaa
EshwarSR
fasiha
fasiha
floresfdev
floresfdev
FrozenTear7
fuwiak
fuwiak
hurick
imraqes
imraqes
imraqes
jducrot
jhdcruz
JoshuaKaden
jwd-ali
jyotiranjanjally
jyotiranjanjally
leosena777
leosena777
leosena777
littlerkap
marco-souza
marco-souza
meteorzero
neotan
neotan
neotan
neotan
neotan
neotan
neotan
neotan
neotan
neotan
nhancv
nhancv
ojacques
olawalle
orsdev
orsdev
orsdev
piyushkumar96
pyboosted
pyboosted
RitikPatni
RitikPatni
RobinL
romanitalian
ronilaukkarinen
simonsnitz
snip77
srad1292
t3chcrazy
thibaultbl
timofeysie
Vasu014
venura9
vixalien
vronney
wahidari
webcat12345
williamtrindade
williamtrindade
yaleksandr89


In [74]:
Corpus

,Name,u_id,bio,Repo_Name,ReadMe,issues,textual_all
0,Valdo,0raclewind,Passionate student learning web development.,"[0raclewind.github.io, bootcamp, cryptogram, C...","[none, b'# Bootcamp\n[![Build Status](https://...",[],Passionate student learning web development.\n...
1,None,1xuan,An Ordinary Person Focusing On Python,"[alien_invasion, blog, CI-CD-test, front-end-p...",[b'# alien_invasion\nA little game developed b...,[],An Ordinary Person Focusing On Python\n none\n...
2,Agustín Tomas Larghi,4gus71n,"Java/Kotlin (mostly Kotlin 😉) dev, mobile engi...","[akrasia, AmazonBooks, AmazonBooksMini, Androi...",[b'# Akrasia\nA django server that I made to c...,[Added HMSPicker hour minute only option #2\n ...,"Java/Kotlin (mostly Kotlin 😉) dev, mobile engi..."
3,Brian L Mayers,4mifix,Brian is a natural hands on technologist. This...,"[action, amber, brooklyn, food, FreeCodeCamp, ...","[none, b'amber\n=====\n1st\n', none, none, b'!...","[Update style.css\n background image, update\n...",Brian is a natural hands on technologist. This...
4,LiNKeR,54LiNKeR,IG(>_<)@54LiNKeR\n* Android Enthusiast(AE)\n* ...,"[54LiNKeR.github.io, android-play-billing, and...",[b'## Welcome to GitHub Pages\nYou can use the...,[Added BubblingImageView and TetrisView\n - [...,IG(>_<)@54LiNKeR\n* Android Enthusiast(AE)\n* ...
...,...,...,...,...,...,...,...
1758,Leonardo Drici,zRelux,Trying to learn or develop something new every...,"[address-book, AziendeExpo, banker-algorithm, ...",[b'# Address-Book\n Address Book system implam...,[@types/jsonwebtokens - Allow Generics for syn...,Trying to learn or develop something new every...
1759,Zundar,zundarik,None,"[202012-otuskotlin-general, dockprom, otuskotl...",[b'# 202012-otuskotlin-general\n\xd0\xa3\xd1\x...,"[Homework 7\n add logging\r\n, Auth\n Сделал а...",None\n none\n Учебный проект по основам Kotlin...
1760,Maria Plieshka,zuni4ka,Learning JavaScript for fun,"[candyshop, code-and-magick, Coding-challenge-...","[none, none, b'This project was bootstrapped w...","[Добавлено имя наставника\n , Update dependenc...",Learning JavaScript for fun\n none\n Graduate ...
1761,Adam Žúrek,zurek11,Student of Slovak Technical University faculty...,"[AVL_socialSite, binary_obfuscation, Company_c...","[none, b'# binary_obfuscation\nBinary obfuscat...",[Added SoftDeleteQuerySet ☔️\n - Removed out o...,Student of Slovak Technical University faculty...


In [77]:
#reader

In [12]:
dt1=Corpus[['u_id','textual_all']]

In [13]:
dt2 = pd.read_csv('C:\GH-Dataset\Git_dt_roles\git_data_single_label.csv')

In [14]:
dt_count_lb = dt2['label'].value_counts()

In [15]:
merged_inner = pd.merge(left=dt1, right=dt2[['gh_login','label']], left_on='u_id', right_on='gh_login')

In [16]:
txt_dt=merged_inner[['textual_all','label']]

In [17]:
txt_dt

,textual_all,label
0,Passionate student learning web development.\n...,Frontend
1,An Ordinary Person Focusing On Python\n none\n...,Backend
2,Brian is a natural hands on technologist. This...,Frontend
3,IG(>_<)@54LiNKeR\n* Android Enthusiast(AE)\n* ...,Mobile
4,:bomb: F.zoomers!!.we?.boomers!!.let {it.go}\n...,Backend
...,...,...
1424,Trying to learn or develop something new every...,Mobile
1425,None\n none\n Учебный проект по основам Kotlin...,Backend
1426,Learning JavaScript for fun\n none\n Graduate ...,Frontend
1427,Student of Slovak Technical University faculty...,Backend


In [18]:
import gensim
from gensim.parsing.preprocessing import *

In [19]:
#store the textual_all in a sperate varaible
documents=txt_dt['textual_all']

In [20]:
#seperte sentences with \n
def split_sentence(corpus):
    sentences = [doc.split("\n") for doc in corpus]
    return sentences

In [21]:
#flat a list of lists into a list
def flatten_list(sentences):
    flatList = [ item for elem in sentences for item in elem]
    return flatList

In [22]:
#tokenize all sentences
def token_in_sentence(list_sentences):
    list_tokens = [list(gensim.utils.tokenize(doc, lower=True)) for doc in list_sentences]
    return list_tokens

In [23]:
#build a bundle of actions to clean data
def custom_filter():
    cstm_filter = [remove_stopwords,split_alphanum, strip_numeric, strip_tags, strip_non_alphanum,stem_text]
    return cstm_filter

In [24]:
#cleaning data based on steps define in custom_filter
def str_preprocessing(tokenize_sentence):
    CUSTOM_FILTERS = custom_filter()
    tokens_new = preprocess_string(" ".join(tokenize_sentence), CUSTOM_FILTERS)
    return tokens_new

In [25]:
#get the possible phrases in the list of tokens
def get_bigram_token(model,tokens):
    bigrams = model[tokens]
    bi_tokens= [sent for sent in bigrams]
    return bi_tokens

In [26]:
#split sentence in documents (Corpus['textual_all'])
sentences_stream = split_sentence(txt_dt['textual_all'])

In [27]:
#tokenize all sentences
tokenize_sentence=[]
for row in sentences_stream:
    tokenize_st = flatten_list(token_in_sentence(row))
    tokenize_sentence.append(tokenize_st)

In [28]:
#cleaning data with costum actions
processing_tokens = [str_preprocessing(sen) for sen in sentences_stream]

In [29]:
clean_tokens=[]
for row in processing_tokens:
    clean_tokens.append([w for w in row if len(w)>2])
#clean_tokens = [w for t in processing_tokens for w in t if len(w) > 1]

In [30]:
#build a bigram model to collect phrases. The threshold equals to 2. 
#It means if two words accure in bigram more than 2 times, then we collect them as a single phrase
#The threshold is small becuase we are working on a sample dataset.
#After changing the sample dataset to real dataset, we should increase the threshold
#min_count: Ignore all words and bigrams with total collected count lower than this value. 5 , 10
#bigram_mdl = gensim.models.phrases.Phrases(tokenize_sentence, min_count=5, threshold=10)
bigram_mdl = gensim.models.phrases.Phrases(clean_tokens, min_count=10, threshold=15)

In [31]:
#Get the phrases in our corpus
tokens_bigram= get_bigram_token(bigram_mdl,clean_tokens)

In [32]:
#bigram model
bi_tmp_df = {'text_all':tokens_bigram,'label':txt_dt['label']}

In [33]:
bi_doc2vec_dt = pd.DataFrame(bi_tmp_df)

In [34]:
bi_doc2vec_dt

,text_all,label
0,"[passion, student, learn, web, develop, static...",Frontend
1,"[ordinari, person, focus, python, littl, game,...",Backend
2,"[brian, natur, hand, technologist, thi, gift, ...",Frontend
3,"[______, drawerarrowdraw, http, github, com, c...",Mobile
4,"[bomb_zoomer, boomer_let, octocat, speech_ball...",Backend
...,...,...
1424,"[try, learn, develop, new, everydai, address, ...",Mobile
1425,"[none, учебный, проект, основам, kotlin, otusk...",Backend
1426,"[learn, javascript, fun, graduat, project, htm...",Frontend
1427,"[student_slovak, technic, univers_faculti, inf...",Backend


In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

In [36]:
doc_tagged = bi_doc2vec_dt.apply(
    lambda r: TaggedDocument(words=r['text_all'], tags=[r.label]), axis=1)

In [37]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [51]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, epochs=20)) for doc in sents])
    return targets, regressors

In [39]:
%%time
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=230, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(doc_tagged.values)])

100%|██████████████████████████████████████████████████████████████████████████| 1429/1429 [00:00<00:00, 477392.31it/s]


Wall time: 10.7 s


In [42]:
%%time
for epoch in range(30):
    model_dmm.train(utils.shuffle([x for x in tqdm(doc_tagged.values)]), total_examples=len(doc_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|█████████████████████████████████████████████████████████████████████████| 1429/1429 [00:00<00:00, 1432176.92it/s]


Wall time: 6min 29s


In [55]:
train, test = train_test_split(doc_tagged, test_size=0.1, random_state=42)

In [56]:
%%time
y_train, X_train = vec_for_learning(model_dmm, train)
y_test, X_test = vec_for_learning(model_dmm, test)


Wall time: 6min 37s


In [48]:
logreg = LogisticRegression(n_jobs=1, C=1e5)

In [57]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

C:\Users\morad\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\morad\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\morad\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [58]:
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print('Percision: {}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Recall: {}'.format(recall_score(y_test, y_pred, average='weighted')))

Accuracy 0.8111888111888111
F1 score: 0.8116871948292954
Percision: 0.814136624192342
Recall: 0.8111888111888111
